# Train your first model

This is the second of our [beginner tutorial series](https://github.com/awslabs/djl/tree/master/jupyter/tutorial) that will take you through creating, training, and running inference on a neural network. In this tutorial, you will learn how to train an image classification model that can recognize handwritten digits.

## Preparation

This tutorial requires the installation of the Java Jupyter Kernel. To install the kernel, see the [Jupyter README](https://github.com/awslabs/djl/blob/master/jupyter/README.md).

In [1]:
%mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.3.0-SNAPSHOT
%maven ai.djl:basicdataset:0.3.0-SNAPSHOT
%maven ai.djl:model-zoo:0.3.0-SNAPSHOT
%maven ai.djl:repository:0.3.0-SNAPSHOT
%maven ai.djl.mxnet:mxnet-engine:0.3.0-SNAPSHOT
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
%maven net.java.dev.jna:jna:5.3.0

### Include MXNet engine dependency

This tutorial uses the MXNet engine as its backend. MXNet has different [build flavors](https://mxnet.apache.org/get_started?version=v1.5.1&platform=linux&language=python&environ=pip&processor=cpu) and it is platform specific.
Please read [here](https://github.com/awslabs/djl/blob/master/examples/README.md#engine-selection) for how to select MXNet engine flavor or keep the recommended CPU flavor below.

In [2]:
String osName = System.getProperty("os.name");
String classifier = osName.startsWith("Mac") ? "osx-x86_64" : osName.startsWith("Win") ? "win-x86_64" : "linux-x86_64";


%maven ai.djl.mxnet:mxnet-native-mkl:jar:${classifier}:1.6.0-c-SNAPSHOT

In [3]:
import java.nio.file.*;

import ai.djl.*;
import ai.djl.basicdataset.*;
import ai.djl.ndarray.types.*;
import ai.djl.training.*;
import ai.djl.training.dataset.*;
import ai.djl.training.initializer.*;
import ai.djl.training.loss.*;
import ai.djl.training.evaluator.*;
import ai.djl.training.optimizer.*;
import ai.djl.training.optimizer.learningrate.*;
import ai.djl.training.util.*;
import ai.djl.basicmodelzoo.cv.classification.*;

# Step 1: Prepare MNIST dataset for training

When training a deep learning network, it is important to first understand the dataset. The dataset we will be using is [MNIST](https://en.wikipedia.org/wiki/MNIST_database), a database of handwritten digits. Each image contains a black and white digit from 0-9 in a 28x28 image. It is commonly used when getting started with deep learning and training image classification models because it is small and fast to train.

Once you understand your dataset, you should create a [Dataset implementation](https://javadoc.djl.ai/api/0.2.1/index.html?ai/djl/training/dataset/Dataset.html). For now, we provide the MNIST dataset built-in, which makes it easy for you to use it. We must also decide the parameters for loading data from the dataset so we will set the batchSize to 32. Batchsizes are usually the largest power of 2 that fits within memory.

In [ ]:
int batchSize = 32;
Mnist mnist = Mnist.builder().setSampling(batchSize, true).build();
mnist.prepare(new ProgressBar());

# Step 2: Create your neural network

In this tutoral, we will use the built-in Multilayer Perceptron from the Model Zoo. To learn more, see the previous tutorial: [Create Your First Network](create_your_first_network.ipynb).

Because images in the MNIST dataset are 28x28 grayscale images, we will create an MLP block with 28 x 28 input.

In [ ]:
Model model = Model.newInstance();
model.setBlock(new Mlp(28, 28));

# Step 3: Setup your training configurations

Before you start training, we we will create a [default training configuration](https://javadoc.djl.ai/api/0.2.1/index.html?ai/djl/training/DefaultTrainingConfig.html) that describes how to train your model.

The following are a few common items you may need to configure your training:
* [`Loss`](https://javadoc.djl.ai/api/0.2.1/index.html?ai/djl/training/loss/Loss.html) function: A loss function is used to measure how good (or bad) our models are. Because the lower value of the function is better, it's called the "loss" function. The Loss is the only required argument to the model
* [`Evaluator`](https://javadoc.djl.ai/api/0.2.1/index.html?ai/djl/training/evaluator/Evaluator.html) function: An evaluator function is also used to measure how good (or bad) our models are. Unlike the loss, they are only there for monitoring and are not optimized. Since many losses are not as intuitive to understand, adding other evaluators such as Accuracy can help to judge your model.
* batch size: To take the advantage of the natural parallelism, you usually train models with batches of input data items rather than a single item at a time. This should match the batch size provided in the model",
* [`Device`](https://javadoc.djl.ai/api/0.2.1/index.html?ai/djl/Device.html): DJL can automatically detect whether a GPU is available. If GPUs are available, it will run on a single GPU by default. If you need to train with multiple GPUs, you need to set devices as : `config.setDevices(Devices.getDevices(maxNumberOfGPUs))`.
* [`Initializer`](https://javadoc.djl.ai/api/0.2.1/index.html?ai/djl/training/initializer/Initializer.html): An `Initializer` initializes model parameters. This can usually be left as the default initializer.
* [`Optimizer`](https://javadoc.djl.ai/api/0.2.1/index.html?ai/djl/training/optimizer/Optimizer.html): Optimization algorithms allow us to continue updating model parameters and to minimize the value of the loss function. When just starting, you can use the default optimizer. Customizing the optimizer can result in faster training.


In [ ]:
TrainingConfig config = new DefaultTrainingConfig(Loss.softmaxCrossEntropyLoss())
        .addEvaluator(new Accuracy())
        .setBatchSize(batchSize);

# Step 4: Create a Trainer

Now, you can create a [`Trainer`](https://javadoc.djl.ai/api/0.2.1/index.html?ai/djl/training/Trainer.html) with your training configuration. The trainer is the main class to orchestrate the training process. Usually, they will be opened using a try-with-resources and closed after training is over.

In [ ]:
Trainer trainer = model.newTrainer(config);

# Step 5: Initialize Training

Before training your model, you have to initialize all of the parameters with default values. You can use the trainer for this initialization by passing in the input shape.

* The first axis of the input shape is the batch size. This won't impact the parameter initialization, so you can use 1 here.
* The second axis of the input shape is the MNIST image size, which is 28 x 28.

In [ ]:
trainer.initialize(new Shape(1, 28 * 28));

# Step 6: Train your model

Now, we can train the model.

In [ ]:
// Setup the progress bar to display the training progress
ProgressBar progressBar = new ProgressBar("Training", (int) mnist.getIteration());

// Deep learning is typically trained in epochs where each epoch trains the model once on each item in the dataset.
int epoch = 2;

float trainingAccuracy = 0f;
for (int i = 0; i < epoch; ++i) {
    int index = 0;
    
    // Each epoch we iterate through the whole dataset once
    for (Batch batch : trainer.iterateDataset(mnist)) {
        
        // During trainBatch, we update the loss and evaluators with the results for the training batch.
        trainer.trainBatch(batch);
        
        // Now, we update the model parameters based on the results of the latest trainBatch
        trainer.step();
        
        // We must make sure to close the batch to ensure all the memory associated with the batch is cleared quickly.
        // If the memory isn't closed after each batch, you will very quickly run out of memory on your GPU
        batch.close();

        trainingAccuracy = accuracy.getValue();
        progressBar.update(index++, String.format("Epoch: %d, Accuracy: %.3f", i, trainingAccuracy));
    }
    // reset training and validation evaluators at the end of the epoch
    trainer.resetEvaluators();
}

// If you did not use a try-with-resources with your trainer, close it once training is over
trainer.close();

# Step 7: Save your model

Once your model is trained, you should save it so that it can be reloaded later. You can also add metadata to it such as training accuracy, number of epochs trained, etc that can be used when loading the model or when examining it.

In [ ]:
Path modelDir = Paths.get("build/mlp");
Files.createDirectories(modelDir);

model.setProperty("Epoch", String.valueOf(epoch));
model.setProperty("Accuracy", String.valueOf(trainingAccuracy));

model.save(modelDir, "mlp");

model

# Summary

Now, you've successfully trained a model that can recognize handwritten digits. You'll learn how to apply this model in the next chapter: [Run image classification with your model](image_classification_with_your_model.ipynb).

You can find the complete source code for this tutorial in the [examples project](https://github.com/awslabs/djl/blob/master/examples/src/main/java/ai/djl/examples/training/TrainMnist.java).